### This file contains helpful code snippets used for minor preprocessing of metadata

### language: Groovy

### Get columns name from columns descriptions

In [2]:
columnsDescriptions = """
title: nazwa łącząca pdb_code (nazwę wpisu do bazy PDB), res_name (nazwę ligandu), res_id, i chain_id;
pbd_code: identyfikator pliku PDB;
res_name: nazwa ligandu, czyli opisanej w wierszu cząsteczki chemicznej (etykieta klasy);
res_id: identyfikator reszty w pliku PDB (umiejscowienie cząsteczki w białku);
chain_id: identyfikator łańcucha w pliku PDB (umiejscowienie cząsteczki w białku);

local_BAa, local_NPa, local_Ra, local_RGa, local_SRGa, local_CCSa, local_CCPa, local_ZOa, local_ZDa, local_ZD_minus_a, local_ZD_plus_a: te zmienne to miary jakości struktury zapisanej w PDB;

local_res_atom_count local_res_atom_non_h_count local_res_atom_non_h_occupancy_sum local_res_atom_non_h_electron_sum local_res_atom_non_h_electron_occupancy_sum local_res_atom_C_count local_res_atom_N_count local_res_atom_O_count local_res_atom_S_count: liczba atomów i elektronów ligandu zamodelowana w pliku PDB;

dict_atom_non_h_count dict_atom_non_h_electron_sum dict_atom_C_count dict_atom_N_count dict_atom_O_count dict_atom_S_count: liczba atomów i elektronów wyliczona na podstawie danych słownikowych (“tablicy pierwiastków”);

part_XX: kolumny zaczynające się od “part_” opisują wartości obliczone tylko na podstawie ligandu; liczba pojawiająca się po “part_” koduje próg odcięcia intensywności; atrybuty obliczone dla każdego poziomu odcięcia zostały opisane poniżej:

blob_electron_sum: szacowana liczba elektronów;
blob_volume_sum: szacowana objętość cząsteczki;
blob_parts: liczba rozdzielnych części;

shape: maska kształtu;
density: ,maska gęstości elektronowej;

O3, O4, O5, FL: niezmienniki kształtu opisane w artykule “Moment invariants as shape recognition technique for comparing protein binding sites”, Sommer et al.
I1, I2, I3, I4, I5, I6: niezmienniki kształtu opisane w artykule “3-D Surface Moment Invariants”, Xu i Li
M000: moment zerowy (licznik potrzebny do obliczeń - brak interpretacji fizycznej);

E1, E2, E3: wartości własne macierzy kowariancji obliczonej dla kształtu i gęstości; E1 to największa wartość własna, E3 najmniejsza; E3_E1 to E3 podzielone przez E1, id., sqrt oznacza pierwiastek danej wartości własnej;

norm: niezmienniki kształtu znormalizowane aby były niezmienne ze względu rotację, translację i skalę;
scaled: wartości niezmienne ze względu rotację, translację, ale nie skalę;

local_volume: objętość dla progu odcięcia 6 x FoFc_std;
local_electrons: liczba elektronów dla progu odcięcia 6 x FoFc_std;
local_mean: średnia gęstość elektronowa dla progu odcięcia 6 x FoFc_std;
local_std: odchylenie standardowe średniej gęstości elektronowej dla progu odcięcia 6 x FoFc_std;
local_min: minimalna gęstość elektronowa dla progu odcięcia 6 x FoFc_std;
local_max: maksymalna gęstość elektronowa dla progu odcięcia 6 x FoFc_std;
local_skewness: skośność dla progu odcięcia 6 x FoFc_std;
local_parts: liczba rozdzielnych części dla progu odcięcia 6 x FoFc_std;

TwoFoFc_mean, TwoFoFc_std TwoFoFc_square_std TwoFoFc_min TwoFoFc_max: to samo co dla local tylko bez progowania (TwoFoFc = 2Fo-Fc);
Fo_mean Fo_std Fo_square_std Fo_min Fo_max: to samo co dla local tylko bez progowania dla mapy Fo (Fo = observed);
FoFc_mean FoFc_std FoFc_square_std FoFc_min FoFc_max: to samo co dla local tylko bez progowania dla mapy FoFc (FoFc = Fo-Fc = error/residual map);
Fc_mean Fc_std Fc_square_std Fc_min Fc_max: to samo co dla local tylko bez progowania dla mapy Fc (Fc = calculated);

fo_col, fc_col, weight_col, grid_space, solvent_radius, solvent_opening_radius: parametry wykorzystane podczas tworzenia zbioru;
resolution: maksymalna rozdzielczość analizowanych danych;

solvent_mask_count: rozmiar maski rozpuszczalnika;
void_mask_count: rozmiar maski pustego obszaru;
modeled_mask_count: rozmiar maski ligandu;
solvent_ratio: solvent/(solvent+void+modeled);

TwoFoFc_bulk_mean, TwoFoFc_bulk_std, TwoFoFc_void_mean, TwoFoFc_void_std, TwoFoFc_modeled_mean, TwoFoFc_modeled_std: globalne statystyki dla mapy TWoFoFc;
Fo_bulk_mean, Fo_bulk_std, Fo_void_mean, Fo_void_std, Fo_modeled_mean, Fo_modeled_std: globalne statystyki dla mapy Fo;
Fc_bulk_mean, Fc_bulk_std, Fc_void_mean, Fc_void_std, Fc_modeled_mean, Fc_modeled_std: globalne statystyki dla mapy Fc;
FoFc_bulk_mean, FoFc_bulk_std, FoFc_void_mean, FoFc_void_std, FoFc_modeled_mean, FoFc_modeled_std: globalne statystyki dla mapy FoFc;
TwoFoFc_void_fit_binormal_mean1, TwoFoFc_void_fit_binormal_std1, TwoFoFc_void_fit_binormal_mean2, TwoFoFc_void_fit_binormal_std2, TwoFoFc_void_fit_binormal_scale, TwoFoFc_solvent_fit_normal_mean, TwoFoFc_solvent_fit_normal_std: dopasowanie do wybranych rozkładów danych;

part_step_FoFc_std_min: minimalny próg maskowania (dla part_00);
part_step_FoFc_std_max: maksymalny próg maskowania (dla part_10);
part_step_FoFc_std_step: krok maskowania (dł/szer./wys. piksela maski).
"""

null

null

In [19]:
matcher = columnsDescriptions =~ /[^:\n]+(?=:)/
colNames = matcher.findAll() as List

[title, pbd_code, res_name, res_id, chain_id, local_BAa, local_NPa, local_Ra, local_RGa, local_SRGa, local_CCSa, local_CCPa, local_ZOa, local_ZDa, local_ZD_minus_a, local_ZD_plus_a, local_res_atom_count local_res_atom_non_h_count local_res_atom_non_h_occupancy_sum local_res_atom_non_h_electron_sum local_res_atom_non_h_electron_occupancy_sum local_res_atom_C_count local_res_atom_N_count local_res_atom_O_count local_res_atom_S_count, dict_atom_non_h_count dict_atom_non_h_electron_sum dict_atom_C_count dict_atom_N_count dict_atom_O_count dict_atom_S_count, part_XX,  kolumny zaczynające się od “part_” opisują wartości obliczone tylko na podstawie ligandu; liczba pojawiająca się po “part_” koduje próg odcięcia intensywności; atrybuty obliczone dla każdego poziomu odcięcia zostały opisane poniżej, blob_electron_sum, blob_volume_sum, blob_parts, shape, density, O3, O4, O5, FL, I1, I2, I3, I4, I5, I6, M000, E1, E2, E3, norm, scaled, local_volume, local_electrons, local_mean, local_std, local_m

In [24]:
formattedColNames = colNames
.collect { "\"$it\"" }
.join ", "

"title", "pbd_code", "res_name", "res_id", "chain_id", "local_BAa, local_NPa, local_Ra, local_RGa, local_SRGa, local_CCSa, local_CCPa, local_ZOa, local_ZDa, local_ZD_minus_a, local_ZD_plus_a", "local_res_atom_count local_res_atom_non_h_count local_res_atom_non_h_occupancy_sum local_res_atom_non_h_electron_sum local_res_atom_non_h_electron_occupancy_sum local_res_atom_C_count local_res_atom_N_count local_res_atom_O_count local_res_atom_S_count", "dict_atom_non_h_count dict_atom_non_h_electron_sum dict_atom_C_count dict_atom_N_count dict_atom_O_count dict_atom_S_count", "part_XX", " kolumny zaczynające się od “part_” opisują wartości obliczone tylko na podstawie ligandu; liczba pojawiająca się po “part_” koduje próg odcięcia intensywności; atrybuty obliczone dla każdego poziomu odcięcia zostały opisane poniżej", "blob_electron_sum", "blob_volume_sum", "blob_parts", "shape", "density", "O3, O4, O5, FL", "I1, I2, I3, I4, I5, I6", "M000", "E1, E2, E3", "norm", "scaled", "local_volume", "loc

### Check content of first lines of the data file

In [26]:
f = new File("all_summary.csv")

all_summary.csv

In [50]:
f.withReader {
    line = it.readLine()
    line2 = it.readLine()
}
names = line.split(";")
namesSize = names.size()
values = line2.split(";")
valuesSize = line2.size()

6110

### Replace pretty quotes with oridenry quotes

In [53]:
colsToDeleteString = "“UNK”, “UNX”, “UNL”, “DUM”, “N”, “BLOB”, “ALA”, “ARG”, “ASN”, “ASP”, “CYS”, “GLN”, “GLU”, “GLY”, “HIS”, “ILE”, “LEU”, “LYS”, “MET”, “MSE”, “PHE”, “PRO”, “SEC”, “SER”, “THR”, “TRP”, “TYR”, “VAL”, “DA”, “DG”, “DT”, “DC”, “DU”, “A”, “G”, “T”, “C”, “U”, “HOH”, “H20”, “WAT”"

“UNK”, “UNX”, “UNL”, “DUM”, “N”, “BLOB”, “ALA”, “ARG”, “ASN”, “ASP”, “CYS”, “GLN”, “GLU”, “GLY”, “HIS”, “ILE”, “LEU”, “LYS”, “MET”, “MSE”, “PHE”, “PRO”, “SEC”, “SER”, “THR”, “TRP”, “TYR”, “VAL”, “DA”, “DG”, “DT”, “DC”, “DU”, “A”, “G”, “T”, “C”, “U”, “HOH”, “H20”, “WAT”

In [70]:
colsToDeleteMatcher = colsToDeleteString =~ /(?<=“)[^“]+(?=”)/
colsToDeleteMatcher.findAll()
.collect { "\"$it\"" }
.join(", ")

"UNK", "UNX", "UNL", "DUM", "N", "BLOB", "ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS", "ILE", "LEU", "LYS", "MET", "MSE", "PHE", "PRO", "SEC", "SER", "THR", "TRP", "TYR", "VAL", "DA", "DG", "DT", "DC", "DU", "A", "G", "T", "C", "U", "HOH", "H20", "WAT"